In [15]:
import json

def get_timestamped_transcript(id, numchunks=3):

    path_to_data_folder = '/archive/shared/sim_center/shared/annie/'
    # lookinto this dictionary to find the path
    # can also manually create the path and it would be faster but not by much

    if numchunks == 3: path = path_to_data_folder + 'diarized-levenshteined-3chunk/' + id + '.json'
    if numchunks == 6: path = path_to_data_folder + 'diarized-levenshteined-6chunk/' + id + '.json'
    if numchunks == 9: path = path_to_data_folder + 'diarized-levenshteined-9chunk/' + id + '.json'

    # Opening file
    # Opening JSON file
    f = open(path)

    # returns JSON object as 
    # a dictionary
    json_transcript = json.load(f)
    
    f.close()

    return json_transcript

In [50]:
id = '10_1005_331402'

def finetune_timestamps(id):

    transcript = get_timestamped_transcript(id)

    threshold = 100 # can change this parameter
    previous = []
    next = transcript[1]['timestamp']
    dels = [] #indices of deleted timestamps

    # artificial approximate first timestamp if it's outlandish
    if transcript[0]['timestamp'][0] > 100:  # can also change/tune this parameter
        transcript[0]['timestamp'][0] = 0.0
        transcript[0]['timestamp'][1] = 100.0


    # iterate through all lines of json except last line as that's usually instructions and comes way later
    for i in range(len(transcript)-1):
        delete = False
        current = transcript[i]['timestamp']
        if previous != [] and None not in previous:
            if current[0] > previous[1] + threshold: delete = True
            if current[1] > previous[1] + threshold: delete = True
            if current[0] < previous[0] - threshold: delete = True
            if current[1] < previous[0] - threshold: delete = True
        '''if next != []:
            if current[0] < next[0] - threshold: delete = True
            if current[1] < next[1] - threshold: delete = True
            if current[0] > next[0] + threshold: delete = True
            if current[1] > next[1] + threshold: delete = True'''
        if delete: 
            transcript[i]['timestamp'] = []
            dels.append(i)

        if not delete: previous = current
        if i != len(transcript) - 1: next = transcript[i+1]['timestamp']
    
    def prior(i):
        if i < 0: return []
        if i > len(transcript): return []

        timestamp = []
        if transcript[i-1]['timestamp'] == []: timestamp = prior(i-1)
        else: timestamp = transcript[i-1]['timestamp']

        return timestamp
    
    def next(i):
        if i < 0: return []
        if i > len(transcript): return []

        timestamp = []
        if transcript[i+1]['timestamp'] == []: timestamp = next(i+1)
        else: timestamp = transcript[i+1]['timestamp']

        return timestamp

    # go through the deleted ones and fill them in with inferred timestamps
    reps = [] # to store replacements
    for i in dels:
        prev = prior(i)[1]
        nex = next(i)[0]
        reps.append([prev, nex])

    for i in range(len(reps)):
        prev = reps[i][0]
        nex = reps[i][1]
        if nex < prev: nex = prev + 40 # arbitrary fix, can change this parameter
        transcript[dels[i]]['timestamp'] = [prev, nex]
          
    return transcript

In [51]:
id = '02_0036_174595'
finetune_timestamps(id)

[{'text': "Student: Thank you. I'm sorry. Are you Mr. Daniel?\n",
  'timestamp': [0.0, 100.0]},
 {'text': 'Patient: Yes.\n', 'timestamp': [100.0, 140.0]},
 {'text': "Student: Hi, Mr Daniel. My name's Kelsey. I am a medical student here in the clinic.\n",
  'timestamp': [0.0, 50.0]},
 {'text': 'Patient: Kelsey, yes.\n', 'timestamp': [50.0, 50.0]},
 {'text': 'Student: Kelsey?\n', 'timestamp': [50.0, 50.0]},
 {'text': "Patient: Kelsey yes, it's right there. \n",
  'timestamp': [50.0, 50.0]},
 {'text': 'Student: Okay. If it is okay with you, I would like to just ask you a few questions about what brought you in today and then do a physical exam.\n',
  'timestamp': [50.0, 66.72]},
 {'text': 'Patient: Sounds good.\n', 'timestamp': [66.72, 117.7]},
 {'text': 'Student: Okay, so what brought you to the clinic today?\n',
  'timestamp': [66.72, 117.7]},
 {'text': 'Patient: I had ECIs and a runny nose.\n',
  'timestamp': [117.7, 119.0]},
 {'text': 'Student: ECI and run a running nose, okay. Okay. 

In [41]:
id_set1 = ['01_0542_298135',
'02_0036_174595',
'03_0028_174553',
'04_0043_174686',
'05_0033_174804',
'06_0079_175106',
'07_0068_174641',
'08_0029_174576',
'09_0029_174582',
'10_0991_331330']

id_set2 = ['01_1080_366142',
           '02_1056_380177',
           '03_1500_380168',
           '04_1512_380182',
           '05_1066_380195',
           '06_1048_365209',
           '07_1111_380134',
           '08_1044_380133',
           '09_1039_380193',
           '10_1005_331402'
]

id_set = id_set1 + id_set2

temp = ['04_1512_380182',
           '05_1066_380195',
           '06_1048_365209',
           '07_1111_380134',
           '08_1044_380133',
           '09_1039_380193',
           '10_1005_331402']

for id in id_set:
    print(id)
    new_trans = finetune_timestamps(id)
    path = '/archive/shared/sim_center/shared/annie/diarized-levenshteined-3chunk-finetuned/'
    with open(path + id + ".json", "w") as outfile:
        json.dump(new_trans, outfile)

01_0542_298135
02_0036_174595
03_0028_174553
04_0043_174686
05_0033_174804
06_0079_175106
07_0068_174641
08_0029_174576
09_0029_174582
10_0991_331330
01_1080_366142
02_1056_380177
03_1500_380168
04_1512_380182
05_1066_380195
06_1048_365209
07_1111_380134
08_1044_380133
09_1039_380193
10_1005_331402
